# Input data and temperature sensitivity

A continuation of the simple sensitivity study we did in 
[Sensitivity studies using COSIPY](sensitivity_study.ipynb) is to investigate how changing the surface temperature, or any other input variable, affects the calculation of the surface mass balance. COSIPY reads the meteorological input data from a **netcdf** file during the run, i.e. during each time step the model reads the meteorological data from the corresponding time step in the input file. The input file can be either "1D" for a point simulation or "2D" for a distributed simulation. The input file can be based on either observed or modeled data.

In order to conduct the temperature sensitivity study we first need to do the following:
- Create the input file from observations.
- Create copies of the input netcdf with different temperature biases.
- Adapt the `NAMELIST` instances to point to the biased files. 

With this done we can run the simulations. But first let's take a look at the input data for Zhadang glacier, which ships with COSPIY.

**The standard imports**

In [ ]:
# Have to change the cwd for the ipython session, otherwise COSIPY
# will look for things in the wrong places.
import os
import sys
# This is not really a good method, if cell is re run we end up in the
# wrong directory.
os.chdir('./../')
sys.path.append(os.getcwd())

In [ ]:
from cosipy.utils import edu_utils
# cfg gives us the NAMELIST
import cfg
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Have to tell matplotlib to plot inline
%matplotlib inline

The NAMELIST already contains the path to the input data, let us open it with `xarray`

In [ ]:
import xarray as xr

In [ ]:
# Path to data
cfg.NAMELIST['data_path']

In [ ]:
# The path of the file
cfg.NAMELIST['input_netcdf']

Open the file with xarray:

In [ ]:
# Use the namelist for path to data and input netcdf. Have to add
# input in between.
input_path = cfg.NAMELIST['data_path'] + 'input/' +\
             cfg.NAMELIST['input_netcdf']
with xr.open_dataset(input_path) as ds:
    ds = ds.isel(time=slice(0, -1)).load()

In [ ]:
ds

As you can see, this file contain variables such as the 2-meter temperature (T2), relative humidity at 2 meters (RH2) and the cloud cover (N) for one single point. The time coordinate spans between 2009-01-01 00:00:00 and 2009-01-31 22:00:00 on a hourly resolution.


<div class="alert alert-warning">
<details>
    <summary><b>Question: Can you figure out what the variable G is and what unit it has?</b> <i>Click me for a hint!</i></summary>
    Try pressing the document symbol to the right in the output above, this shows some extra information about the variable.
    </details>
</div>

We can quickly take a look at one of the variables

In [ ]:
ds.T2.plot(figsize=(10,5));

## Creating the input files

In this case COSIPY comes packaged with the processed data which can directly be used to drive a simulation. In a more realistic scenario however, you probably want drive COSIPY with your own data on another glacier than the Zhadang glacier. The `aws2cosipy` module is provided by COSIPY to aid the processing of .csv-files from weather stations into input files which can be used by COSIPY. 

In [ ]:
# We import the create_1D_input function to process a single point
from utilities.aws2cosipy.aws2cosipy import create_1D_input

The `create_1D_input` has five arguments: the csv file to process, the name of the resulting input file, the name of a static file describing the altitude, slope and aspect of the point, and the start and end date.

The .csv file is located in the same directory as the input file we looked at earlier. It is called `Zhadang_ERA5_2009_2018.csv`. We can process it

In [ ]:
# Path to the file
data_folder = cfg.NAMELIST['data_path']
file = data_folder + 'input/Zhadang/Zhadang_ERA5_2009_2018.csv'

In [ ]:
# Static file
static_file = data_folder + 'static/Zhadang_static.nc'

In [ ]:
# Output file
output_name = data_folder + 'input/Zhadang/Zhadang_ERA5_2009.nc'

In [ ]:
# Define time start and end.
start_date = '20090101'
end_date = '20090131'

In [ ]:
# Call the function, this takes some time.
create_1D_input(file, output_name, static_file, start_date, end_date)

Open the file to see that it worked

In [ ]:
with xr.open_dataset(output_name) as ds:
    ds = ds.isel(time=slice(0, -1)).load()

In [ ]:
ds

This basically recreated the file that we already had, but feel free to try it on another time period. The .csv-file contains data from January 2000 until early January 2018.

## Creating datasets for temperature bias experiments

A temperature bias experiment is essentially a sensitivity study. It explores the effects of changing the temperature by $n$ degrees over the whole time period, let's say we want to know how the glacier responds to a climate which is  2 °C warmer. However this approach is quite rough since it doesn't take other changes that might come with a warmer climate into account.

Setting this up with COSIPY follows a similar approach as described in the [Sensitivity studies with COSIPY](sensitivity_study.ipynb). However, instead of manipulating the namelist we are changing copies of the dataset containing the input data before sending it to the `cosipy_core`.

We begin by initiating the IOClass and datasets

In [ ]:
# Default
IO_def, DATA_def, RESULTS_def = edu_utils.create_IO(cfg.NAMELIST)
# Up
IO_up, DATA_up, RESULTS_up = edu_utils.create_IO(cfg.NAMELIST)
# Down
IO_dn, DATA_dn, RESULTS_dn = edu_utils.create_IO(cfg.NAMELIST)

And then apply the bias to the input data

In [ ]:
# Bias is two degrees
bias = 2
DATA_dn['T2'] = DATA_dn['T2'] - bias
DATA_up['T2'] = DATA_up['T2'] + bias

**Plot it to make sure it worked**

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
DATA_def['T2'].plot(ax=ax, label='Default')
DATA_up['T2'].plot(ax=ax, label='+2')
DATA_dn['T2'].plot(ax=ax, label='-2')
plt.legend();

### Setting up the run
As in the previous sensitivity study we put everything in a list of lists. Note that we're not changing the namelist this time, only the input data and results dataset.

In [ ]:
# List of lists with our experiments
exp_list = [[cfg.NAMELIST, DATA_def, RESULTS_def, IO_def],
            [cfg.NAMELIST, DATA_dn, RESULTS_dn, IO_dn],
            [cfg.NAMELIST, DATA_up, RESULTS_up, IO_up]
           ]

And lets runs the experiments

In [ ]:
for exp in exp_list:
    # Call run_model once for each experiment
    edu_utils.run_model(exp[1], exp[3], exp[0], exp[2])

And we can re-use the snippet of code to plot the results

In [ ]:
labels = ['Default', '-2$\degree$C', '+2$\degree$C']
fig, ax = plt.subplots(figsize=(10, 5))
for exp, label in zip(exp_list, labels):
    # Get the data and plot it RESULTS are kept at the third spot, index 2
    exp[2].surfMB.plot(ax=ax, label=label)
plt.legend(); 

<div class="alert alert-warning">
    <details>
        <summary>
            <b>Question: Do you understand the plot above?</b> <i>Click me for an explanation</i>
        </summary>
        Similarly to the albedo experiment we did in an earlier notebook changing the temperature directly affects the surface mass balance. Lowering the temperature (negative bias) leads to less mass loss during the day, while increasing the temperature results in increased mass loss.
    </details>
</div>

## How the surface mass balance is calculated

We have now seen how changing the surface temperature affects the calculation of  surface mass balance on our glacier. To understand this a bit better we will go through how COSIPY is calculating the surface mass balance. Lets start from the top.

The surface mass balance, which is measured in m w.e. (meter water equivalent),  is the result of adding the snowfall ($SF$) and deposition ($D$) together and removing the melt ($M$), the sublimation ($s$) and the evaporation/condensation ($e$)

$$
MB_{surf} = SF + D - M - s - e.
$$

Note that the evaporation/condensation depends on the sign. We can plot the variables from our `RESULTS` dataset

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
mb_vars = ['SNOWFALL', 'surfM', 'SUBLIMATION', 'DEPOSITION', 'EVAPORATION',
           'CONDENSATION']
for var in mb_vars:
    RESULTS_def[var].plot(ax=ax, label=var)
ax.set_ylabel('m w.e.')    
plt.legend();

For our simulated period the plot shows us that sublimation and snowfall make up the main part of the changing mass balance. But there is also a small contribution from deposition. Others are negligible.

### Snowfall
**Snowfall** can be a direct product of the weather station/gcm data. In this case it is only multiplied by a constant (`mult_factor_RRR`) and then used directly in the surface mass balance calculation. 

In the case when only the rain rates are provided **snowfall** is derived using a logistic transfer function

$$
\mathrm{SNOWFALL} = \frac{RRR}{1000.0} \frac{\rho_{ice}}{\rho_{fresh\space snow}} \frac{-tanh((T2 - t_0) - c_1) \cdot c_2) + 1}{2}
$$

where $c_1$ and $c_2$ are the center and spread transfer functions. This smoothly scales the solid precipitation from 100% at $t_0$ and 0% at 2 °C. The density of fresh snow is calculated as a function of the air temperature and wind velocity.

### Melt

The surface melt is calculated from the energy available for melt ($q_m$) and the latent heat required for melting ice ($L_s$)

$$
\mathrm{M} = \frac{q_m \cdot dt}{L_s \cdot 1000}.
$$

We multiply with dt to get the total energy available during the time step, and divide with $1000$ to convert the result to m w.e.

The energy available for melt is the sum of the radiative fluxes:
- Net short wave radiation
- Net long wave radiation
- Ground heat flux
- Rain heat flux 
- Sensible heat flux
- Latent heat flux

These are available in the `RESULTS` as well so we can take a look at them

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

(RESULTS_def['LWin'] + RESULTS_def['LWout']).plot(ax=ax, label='LW net')
# Short wave net is actually not in the Results but we can calculate it
(RESULTS_def['G'] * (1 - RESULTS_def['ALBEDO'])).plot(ax=ax, label='SW net')
RESULTS_def['H'].plot(ax=ax, label='Sensible heat flux')
RESULTS_def['LE'].plot(ax=ax, label='Latent heat flux')
RESULTS_def['B'].plot(ax=ax, label='Ground heat flux')
RESULTS_def['QRR'].plot(ax=ax, label='Rain heat flux')
ax.set_ylabel('Energy [W m$^{-2}$]')
plt.legend();

This plot contains a lot of information but there are some key points that you should notice
- During the day, the **net short wave** radiation (orange), which act to heat the surface of the glacier is largely compensated by the negative **ground heat flux**. This means that energy is transported into the glacier, slowly heating it.
- During the night, the sign of the **ground heat flux** is reversed when the glacier release energy in the form of outgoing **long wave radiation**.

From the look of it, the net energy of the variables above should be close to zero. Let's take a look

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
RESULTS_def['ME'].plot(ax=ax);

With this in mind we can dissect the radiative fluxes. 

#### Net shortwave radiation and albedo
The **net shortwave radiation** is defined as
$$
q_{sw} = (1-\alpha) \cdot q_G
$$

where $q_G$ is the incoming shortwave radiation and $\alpha$ is the **albedo**. $q_G$ is given by the input data, while the evolution of the albedo is parametrized according to the approach from Oerlemans and Knap ([1998](https://www.cambridge.org/core/journals/journal-of-glaciology/article/1-year-record-of-global-radiation-and-albedo-in-the-ablation-zone-of-morteratschgletscher-switzerland/F30150EB1AD9D62A0C007C22FFAE7B6A)). In it, the snow albedo is based on the time since the last snowfall ($s$) and an albedo timescale ($\tau^*$) that specifies the speed of the degradation of the albedo from that fresh snow ($\alpha_s$) to that of firn ($\alpha_f$)

$$
\alpha_{snow} = \alpha_f + (\alpha_s - \alpha_f)\space exp \left(\frac{s}{\tau^*}\right).
$$

As the thickness of the snowpack ($d$) decreases, the surface albedo should approach that of the albedo of ice ($\alpha_i$). This is done by introducing a characteristic snow depth scale ($d^*$) to employ what is known as e-folding. With this, the full surface albedo can be written as 

$$
\alpha = \alpha_s + (\alpha_i - \alpha_s)\space exp \left(\frac{-d}{d^*}\right).
$$

The surface albedo is reset to that of fresh snow when new snow accumulation exceeds a certain threshold.

The fraction of the net shortwave radiation that penetrates the glacier decays with the depth ($z$) from the surface according to

$$
Q(z, t) = \lambda_r q_{sw} \space exp(-z \beta)
$$

where $\lambda_r$ is the fraction of absorbed radiation and $\beta$ the extinction coefficient (both depending on if the integrated depth is snow or ice). 

<div class="alert alert-success">
   The constants in the equation above are all available in the NAMELIST. Remember that we changed one of them, the albedo of fresh snow, in the sensitivity study. 
</div>

#### Net longwave radiation

The **net longwave radiation** can be handled in two different ways:
If the input data contains the incoming longwave radiation, the net is calculated as

$$
q_{lw} = q_{lw_{in}} - \epsilon_s \sigma T_0^4,
$$

where $\epsilon_s$ is the surface emissivity, in our case constant $\sim 1$.

If the input data doesn't contain the incoming longwave radiation directly, it  will be parametrized using the Stefan-Boltzman law with the means of the air temperature ($T_{zt}$) and the atmospheric emissivity

$$
\epsilon_a = \epsilon_{cs}(1-N^2) + \epsilon_{cl}N^2,
$$

where $N$ is the cloud cover fraction and $\epsilon_{cs}$ and $\epsilon_{cl}$ are the clear-sky and cloud emissivity. The cloud emissivity is kept constant while the clear-sky emissivity is given by

$$\epsilon_{cs} = 0.23 + 0.433(e_{zt}/T_{zt})^{1/8}.$$

where $e_{zt}$ is the water vapor pressure.

### Sublimation



## Next steps
[Back to overview](welcome.ipynb)

[Distributed simulations](distributed_run.ipynb)
